# Question Answer
- Index
    - Create an index
    - Query the index directly with LLM
    - Query the index via Chain
- Vector Store
    - Create a vector store
    - Query the vector store directly with LLM
    - Query the vector store via Chain
- Chain Type

---

## Setup

In [1]:
import openai
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
openai.api_type = os.environ.get("OPENAI_API_TYPE")
openai.api_base = os.environ.get("OPENAI_API_BASE")
openai.api_key = os.environ.get("OPENAI_API_KEY")
openai.api_version = os.environ.get("OPENAI_API_VERSION")

In [2]:
from langchain.chat_models import AzureChatOpenAI

llm = AzureChatOpenAI(
    deployment_name="gpt4",
    temperature=0,
)

## Index

### Create an index
We create index when the documents are not small and we need to break them into smaller chunks. This is done to improve the performance of the search.

In [4]:
from langchain.document_loaders import CSVLoader

file = "../../data/OutdoorClothingCatalog_1000_small.csv"
loader = CSVLoader(file_path=file)
docs = loader.load()

In [5]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.embeddings import HuggingFaceEmbeddings

index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch, embedding=HuggingFaceEmbeddings()
).from_loaders([loader])

### Query the index directly with LLM
Fetches all the relevant documents from the index and then sends them as context to the LLM model.

In [6]:
from IPython.display import display, Markdown

query = "Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

response = index.query(query, llm)
display(Markdown(response))

| Name                          | Sun Protection | Description                                                                                                   |
|-------------------------------|----------------|---------------------------------------------------------------------------------------------------------------|
| Performance Plus Woven Shirt  | UPF 40+        | Breathable, quick-drying, moisture-wicking, and abrasion-resistant; perfect for trail or travel.              |
| Women's Tropical Plaid Shirt  | UPF 50+        | Lightweight, moisture-wicking, wrinkle-resistant, and ventilated; designed for hot weather and flattering fit. |

**Performance Plus Woven Shirt**: This shirt is perfect for trail or travel, featuring a breathable and quick-drying fabric. It has a slightly fitted design that softly shapes the body and falls at the hip. The fabric is 100% nylon and provides UPF 40+ sun protection. It is also abrasion-resistant for exceptional durability.

**Women's Tropical Plaid Shirt**: Designed for hot weather, this lightweight shirt offers a flattering fit and keeps you cool and comfortable by wicking away perspiration. It is made of 52% polyester and 48% nylon, providing UPF 50+ sun protection. The shirt is wrinkle-resistant and features front and back cape venting for ventilation, as well as low-profile pockets and side shaping for a more flattering fit.

### Query the index via Chain
Fetches all the relevant documents from the index and then sends them as context to the LLM model.

In [7]:
from langchain.chains import RetrievalQA

qa_stuff = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=index.vectorstore.as_retriever(),
    verbose=True,
)

response = qa_stuff.run(query)
display(Markdown(response))



> Entering new  chain...

> Finished chain.


| Name                          | Sun Protection | Description                                                                                                   |
|-------------------------------|----------------|---------------------------------------------------------------------------------------------------------------|
| Performance Plus Woven Shirt  | UPF 40+        | Breathable, quick-drying, moisture-wicking, and abrasion-resistant; perfect for trail or travel.              |
| Women's Tropical Plaid Shirt  | UPF 50+        | Lightweight, moisture-wicking, wrinkle-resistant, and ventilated; designed for hot weather and flattering fit. |

**Performance Plus Woven Shirt**: This shirt is perfect for trail or travel, featuring a breathable and quick-drying fabric. It has a slightly fitted design that softly shapes the body and falls at the hip. The fabric is 100% nylon and provides UPF 40+ sun protection. It is also abrasion-resistant for exceptional durability.

**Women's Tropical Plaid Shirt**: This lightweight shirt is designed for hot weather and has a flattering fit. It is made of 52% polyester and 48% nylon, providing UPF 50+ sun protection. The shirt keeps you cool and comfortable by wicking perspiration away from your skin and dries in minutes. It is also wrinkle-resistant and features front and back cape venting for ventilation.

## Vector Store

### Create a vector store directly from the documents
When the documents are small, we don't need indexing. We can directly create a vector store from the documents.

In [8]:
db = DocArrayInMemorySearch.from_documents(
    documents=docs, embedding=HuggingFaceEmbeddings()
)

### Query the vector store directly with LLM
Fetch all the relevant documents from the vector store and then send them as context to the LLM model.

In [9]:
# embed = embeddings.embed_query(query)
docs = db.similarity_search(query)
docs

[Document(page_content=": 28\nname: Performance Plus Woven Shirt\ndescription: Perfect for trail or travel, this breathable summer shirt has the look and feel of cotton - but is packed with performance. Size & Fit Slightly Fitted: Softly shapes the body. Falls at hip. \r\n\r\nWhy We Love It Designed for total versatility in the backcountry or city streets. We made this woven top with a special quick-dry fabric – one of the fastest drying you'll find anywhere, so you can hit the trail or town in a top that looks great and performs even better. \r\n\r\nFabric & Care 100% nylon. UPF 40+ rated to block the sun's UV rays. Machine wash and dry. \r\n\r\nAdditional Features Dries in less than fourteen minutes. Wicks away moisture. Abrasion-resistant for exceptional durability. Imported. \r\n\r\nSun Protection That Won't Wear Off Our high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun's harmful rays.", metadata={'source': '../../data/OutdoorClothingCatalog_1000_smal

In [16]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])

delimeter = "````"
prompt = f"""
context: {delimeter}{qdocs}{delimeter}
Using the context mentioned above answer the question below.
question: {delimeter}{query}{delimeter}
"""

response = llm.call_as_llm(prompt)
display(Markdown(response))

| Name                          | Description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
|-------------------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Performance Plus Woven Shirt  | This slightly fitted shirt is perfect for trail or travel, made with 100% nylon and UPF 40+ rated fabric to block the sun's UV rays. It is designed for versatility, with quick-dry fabric that dries in less than fourteen minutes, wicks away moisture, and is abrasion-resistant for exceptional durability.                                                                                                                                                                   |
| Women's Tropical Plaid Shirt  | This lightweight hot-weather shirt has a slightly fitted design that softly shapes the body and falls at the hip. Made with 52% polyester and 48% nylon, it has a UPF 50+ rating for the highest sun protection possible. The shirt keeps you cool and comfortable by wicking perspiration away from your skin and drying in minutes. It is wrinkle-resistant and features front and back cape venting for ventilation, low-profile pockets, and side shaping for a flattering fit. |

### Query the vector store via Chain
Fetches all the relevant documents from the vector store and then sends them as context to the LLM model.

In [11]:
from langchain.chains import RetrievalQA

qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=db.as_retriever(), verbose=True
)

response = qa_stuff.run(query)
display(Markdown(response))



> Entering new  chain...

> Finished chain.


| Name                          | Sun Protection | Description                                                                                                   |
|-------------------------------|----------------|---------------------------------------------------------------------------------------------------------------|
| Performance Plus Woven Shirt  | UPF 40+        | Breathable, quick-dry fabric, slightly fitted, falls at hip, abrasion-resistant, and moisture-wicking.        |
| Women's Tropical Plaid Shirt  | UPF 50+        | Lightweight, slightly fitted, falls at hip, moisture-wicking, wrinkle-resistant, and front and back venting. |

## Chain Type

### `stuff`
<br/>
<img src="../../images/chain-type-stuff.png" alt="Chain Type: Stuff" style="width: 60%; height: auto;"/>
<br/>

- Simplest method.
- Simply stuff all data into the prompt as context to pass to the LLM model.

Pros:
- Makes a single call to the LLM model.
- The LLM model has access to all the data at once.

Cons:
- LLMs have a context length limit, and for large documents or many documents this will not work as it will result in a prompt larger than the context length limit.

### Additional Chain Types
<br/>
<img src="../../images/chain-type-additional.png" alt="Additional Chain Types" style="width: 80%; height: auto;"/>

1. **Map_reduce**: Takes chunks one by one, sends them to the LLM model one by one, receives the individual responses. And then makes a final call to the LLM model to summarize the individual responses. 

    Pros:
    - It can operate on any number of documents, as each chunk is sent to the LLM model individually.
    - It can do the individual questions in parallel.

    Cons:
    - It does take a lot more call.
    - It treats each document independently, so it doesn't have access to all the data at once, which may not always be the most desired thing.
    
    <br/>

2. **Refine**: Used to loop over many documents, but this happens iteratively. It builds upon the answer from the previous document.

    Pros:
    - It can operate on any number of documents, as each chunk is sent to the LLM model individually.
    - Really good for combining information and building up an answer over time.

    Cons:
    - It takes the same number of calls as Map_Reduce.
    - Generally lead to longer answers.
    - It's not that fast because now the calls aren't independent, they depend on the result of previous calls.

    <br/>

3. **Map_rerank**: Single call to the LLM for each document, and it also returns a score. And then you select the highest score. This relies on the LLM to know what the score should be, so prompt-engineering is required to tell the LLM that it should be high score if it's releveant to the document.

    Pros:
    - It can operate on any number of documents, as each chunk is sent to the LLM model individually.
    - It can do the individual questions in parallel.

    Cons:
    - It takes the same number of calls as Map_Reduce.
    - It's not always the most relevant answer.

> Note: These methods (chain types) can also be used for lots of other chains besides just question answering. For example, a really common use case of the Map_reduce chain type is summarization, where you have a really long document, and you want to recursively summarize it.